In [10]:
import pygame
import random

# 초기 설정
pygame.init()
WIDTH, HEIGHT = 800, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("🔥 슈퍼 공룡 러너")
clock = pygame.time.Clock()
font = pygame.font.SysFont("consolas", 30)

# 색상
WHITE = (255, 255, 255)
GROUND_COLOR = (235, 222, 200)
DINO_COLOR = (100, 100, 255)
OBSTACLE_COLOR = (34, 139, 34)
CLOUD_COLOR = (255, 255, 255)
BUTTON_COLOR = (255, 255, 255)
BUTTON_OUTLINE = (200, 200, 200)
BUTTON_TEXT = (50, 50, 50)
ITEM_COLOR = (255, 223, 0)
PARTICLE_COLOR = (255, 100, 100)

GROUND_Y = 300

# 공룡
dino = pygame.Rect(80, GROUND_Y - 50, 50, 50)
dino_vel_y = 0
is_jumping = False
is_boosted = False
boost_timer = 0

# 게임 속도
game_speed = 9

# 장애물
obstacles = []
obstacle_timer = 0

# 아이템
items = []
item_timer = 0

# 구름
clouds = []

# 점수
score = 0
high_score = 0

# 배경 스크롤용
ground_x = 0

# 게임 상태
game_active = True

# 파티클 효과
particles = []

# 버튼
restart_btn = pygame.Rect(WIDTH // 2 - 80, HEIGHT // 2 + 40, 160, 50)

# 하늘 색 바뀌기
def get_sky_color(score):
    if score < 20:
        return (135, 206, 250)
    elif score < 50:
        return (250, 160, 100)
    else:
        return (30, 30, 60)

# 버튼 그리기
def draw_button():
    pygame.draw.rect(screen, BUTTON_OUTLINE, restart_btn.inflate(6, 6), border_radius=10)
    pygame.draw.rect(screen, BUTTON_COLOR, restart_btn, border_radius=10)
    text = font.render("다시 시작", True, BUTTON_TEXT)
    screen.blit(text, (restart_btn.x + 20, restart_btn.y + 10))

# 장애물 패턴
def spawn_obstacle_pattern():
    pattern = random.choice(["single", "double", "lowhigh", "bird"])
    obs_list = []
    if pattern == "single":
        obs_list.append(pygame.Rect(WIDTH, GROUND_Y - 40, 25, 40))
    elif pattern == "double":
        obs_list.append(pygame.Rect(WIDTH, GROUND_Y - 40, 25, 40))
        obs_list.append(pygame.Rect(WIDTH + 40, GROUND_Y - 40, 25, 40))
    elif pattern == "lowhigh":
        obs_list.append(pygame.Rect(WIDTH, GROUND_Y - 20, 30, 20))
        obs_list.append(pygame.Rect(WIDTH + 60, GROUND_Y - 80, 30, 30))
    elif pattern == "bird":
        obs_list.append(pygame.Rect(WIDTH, GROUND_Y - 100, 40, 30))
    return obs_list

# 게임 리셋
def reset_game():
    global dino, dino_vel_y, is_jumping, obstacles, obstacle_timer, score, game_speed
    global items, item_timer, is_boosted, boost_timer, particles
    dino.y = GROUND_Y - 50
    dino_vel_y = 0
    is_jumping = False
    obstacles.clear()
    obstacle_timer = 0
    items.clear()
    item_timer = 0
    score = 0
    game_speed = 9
    is_boosted = False
    boost_timer = 0
    particles.clear()
    clouds.clear()

# 파티클 생성
def spawn_particles(x, y):
    for _ in range(5):
        particles.append([x, y, random.randint(-3, 3), random.randint(-5, -1), random.randint(5, 15)])

# 게임 루프
running = True
while running:
    dt = clock.tick(60)
    screen.fill(get_sky_color(score))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if not game_active and event.type == pygame.MOUSEBUTTONDOWN:
            if restart_btn.collidepoint(event.pos):
                game_active = True
                reset_game()

    keys = pygame.key.get_pressed()

    if game_active:
        if keys[pygame.K_SPACE] and not is_jumping:
            dino_vel_y = -15
            is_jumping = True

        dino.y += dino_vel_y
        dino_vel_y += 1
        if dino.y >= GROUND_Y - dino.height:
            dino.y = GROUND_Y - dino.height
            dino_vel_y = 0
            is_jumping = False

        ground_x -= game_speed
        if ground_x <= -WIDTH:
            ground_x = 0
        pygame.draw.rect(screen, GROUND_COLOR, (ground_x, GROUND_Y, WIDTH * 2, HEIGHT - GROUND_Y))

        if random.randint(0, 100) < 2:
            cloud = pygame.Rect(WIDTH, random.randint(50, 150), 60, 20)
            clouds.append(cloud)
        for cloud in clouds[:]:
            cloud.x -= game_speed // 2
            pygame.draw.ellipse(screen, CLOUD_COLOR, cloud)
            if cloud.x + cloud.width < 0:
                clouds.remove(cloud)

        obstacle_timer += 1
        if obstacle_timer > 80:
            obstacle_timer = 0
            obstacles.extend(spawn_obstacle_pattern())
        for obs in list(obstacles):
            obs.x -= game_speed
            pygame.draw.rect(screen, OBSTACLE_COLOR, obs)
            if obs.colliderect(dino) and not is_boosted:
                game_active = False
                high_score = max(high_score, score)
            if obs.x + obs.width < 0:
                obstacles.remove(obs)
                score += 1
                if score % 10 == 0:
                    game_speed += 1

        item_timer += 1
        if item_timer > 200:
            item_timer = 0
            items.append(pygame.Rect(WIDTH, random.randint(GROUND_Y - 100, GROUND_Y - 30), 20, 20))
        for item in list(items):
            item.x -= game_speed
            pygame.draw.rect(screen, ITEM_COLOR, item)
            if item.colliderect(dino):
                is_boosted = True
                boost_timer = 120
                items.remove(item)
                spawn_particles(dino.centerx, dino.centery)
            elif item.x + item.width < 0:
                items.remove(item)

        if is_boosted:
            pygame.draw.rect(screen, (255, 100, 100), dino.inflate(10, 10), 3)
            boost_timer -= 1
            if boost_timer <= 0:
                is_boosted = False

        pygame.draw.rect(screen, DINO_COLOR, dino)

        for p in list(particles):
            p[0] += p[2]
            p[1] += p[3]
            p[4] -= 1
            pygame.draw.circle(screen, PARTICLE_COLOR, (p[0], p[1]), 3)
            if p[4] <= 0:
                particles.remove(p)

        score_text = font.render(f"점수: {score}", True, (0, 0, 0))
        screen.blit(score_text, (10, 10))

    else:
        game_over_text = font.render("게임 오버!", True, (255, 80, 80))
        score_text = font.render(f"점수: {score} / 최고기록: {high_score}", True, (0, 0, 0))
        screen.blit(game_over_text, (WIDTH // 2 - 80, HEIGHT // 2 - 50))
        screen.blit(score_text, (WIDTH // 2 - 130, HEIGHT // 2 - 10))
        draw_button()

    pygame.display.update()

pygame.quit()
